# Detecção de Fósseis com Redes Neurais

Pipeline para detecção automática de esponjas Leptomitid usando:

1. **SAM (Segment Anything)** - Segmentação zero-shot
2. **Grounding DINO + SAM** - Detecção open-vocabulary
3. **Claude Vision API** - Detecção via LLM multimodal
4. **YOLOv8** - Fine-tuning com poucas amostras

---

## 1. Setup

In [ ]:
# Instalar dependências
# Descomente conforme necessário

# SAM
# !pip install segment-anything
# !pip install opencv-python matplotlib

# Grounding DINO + SAM (via groundingdino)
# !pip install groundingdino-py

# YOLOv8
# !pip install ultralytics

# Para Claude API
# !pip install anthropic

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import json
import torch
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

plt.rcParams['figure.figsize'] = (14, 10)

In [ ]:
@dataclass
class Detection:
    """Detecção de fóssil"""
    bbox: Tuple[int, int, int, int]  # x1, y1, x2, y2
    mask: Optional[np.ndarray]
    confidence: float
    label: str = "fossil_sponge"


def load_image(path: str) -> np.ndarray:
    img = cv2.imread(str(path))
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


def visualize_detections(img: np.ndarray, detections: List[Detection], title: str = ""):
    """Visualiza detecções na imagem"""
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Original com bboxes
    vis = img.copy()
    for i, det in enumerate(detections):
        x1, y1, x2, y2 = det.bbox
        cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(vis, f"#{i} {det.confidence:.2f}", (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    axes[0].imshow(vis)
    axes[0].set_title(f"{title} - Bounding Boxes")
    axes[0].axis('off')
    
    # Máscaras combinadas
    if any(det.mask is not None for det in detections):
        overlay = img.copy().astype(float)
        colors = plt.cm.rainbow(np.linspace(0, 1, len(detections)))
        
        for det, color in zip(detections, colors):
            if det.mask is not None:
                mask_3d = np.stack([det.mask]*3, axis=-1)
                overlay = np.where(mask_3d, overlay * 0.5 + np.array(color[:3]) * 255 * 0.5, overlay)
        
        axes[1].imshow(overlay.astype(np.uint8))
        axes[1].set_title(f"{title} - Segmentation Masks")
    else:
        axes[1].imshow(vis)
        axes[1].set_title("No masks available")
    
    axes[1].axis('off')
    plt.tight_layout()
    plt.show()

---
## 2. SAM (Segment Anything Model)

Meta's SAM é um modelo de segmentação zero-shot. Funciona com prompts:
- **Pontos**: Clique no objeto
- **Bounding Box**: Caixa aproximada
- **Automático**: Segmenta tudo na imagem

### Download do modelo:
```bash
# SAM ViT-H (2.4GB) - mais preciso
wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

# SAM ViT-B (375MB) - mais rápido
wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
```

In [ ]:
class SAMDetector:
    """
    Detector usando Segment Anything Model
    """
    
    def __init__(self, 
                 checkpoint_path: str = "sam_vit_b_01ec64.pth",
                 model_type: str = "vit_b",
                 device: str = DEVICE):
        
        from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
        
        print(f"Carregando SAM ({model_type})...")
        self.sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
        self.sam.to(device)
        
        self.predictor = SamPredictor(self.sam)
        self.auto_generator = SamAutomaticMaskGenerator(
            self.sam,
            points_per_side=32,
            pred_iou_thresh=0.86,
            stability_score_thresh=0.92,
            min_mask_region_area=10000,  # Filtrar regiões pequenas
        )
        
        print("SAM carregado!")
    
    def segment_with_points(self, 
                            img: np.ndarray,
                            points: List[Tuple[int, int]],
                            labels: List[int] = None) -> List[Detection]:
        """
        Segmenta usando pontos como prompt.
        
        Args:
            img: Imagem RGB
            points: Lista de (x, y) indicando objetos
            labels: 1 para foreground, 0 para background
        """
        self.predictor.set_image(img)
        
        points_np = np.array(points)
        labels_np = np.array(labels) if labels else np.ones(len(points))
        
        masks, scores, _ = self.predictor.predict(
            point_coords=points_np,
            point_labels=labels_np,
            multimask_output=True
        )
        
        # Pegar a melhor máscara
        best_idx = np.argmax(scores)
        best_mask = masks[best_idx]
        
        # Calcular bbox
        coords = np.where(best_mask)
        if len(coords[0]) > 0:
            y1, y2 = coords[0].min(), coords[0].max()
            x1, x2 = coords[1].min(), coords[1].max()
            bbox = (x1, y1, x2, y2)
        else:
            bbox = (0, 0, 0, 0)
        
        return [Detection(
            bbox=bbox,
            mask=best_mask,
            confidence=float(scores[best_idx])
        )]
    
    def segment_with_box(self, 
                         img: np.ndarray,
                         box: Tuple[int, int, int, int]) -> List[Detection]:
        """
        Segmenta usando bounding box como prompt.
        
        Args:
            img: Imagem RGB
            box: (x1, y1, x2, y2) aproximado do objeto
        """
        self.predictor.set_image(img)
        
        masks, scores, _ = self.predictor.predict(
            box=np.array(box),
            multimask_output=True
        )
        
        best_idx = np.argmax(scores)
        best_mask = masks[best_idx]
        
        # Refinar bbox baseado na máscara
        coords = np.where(best_mask)
        if len(coords[0]) > 0:
            y1, y2 = coords[0].min(), coords[0].max()
            x1, x2 = coords[1].min(), coords[1].max()
            refined_bbox = (x1, y1, x2, y2)
        else:
            refined_bbox = box
        
        return [Detection(
            bbox=refined_bbox,
            mask=best_mask,
            confidence=float(scores[best_idx])
        )]
    
    def segment_automatic(self, 
                          img: np.ndarray,
                          min_area: int = 10000,
                          max_area: int = None,
                          aspect_ratio_range: Tuple[float, float] = (1.5, 10.0)) -> List[Detection]:
        """
        Segmentação automática com filtros para fósseis.
        
        Args:
            img: Imagem RGB
            min_area: Área mínima em pixels
            max_area: Área máxima (None = 50% da imagem)
            aspect_ratio_range: Fósseis Leptomitid são alongados
        """
        if max_area is None:
            max_area = img.shape[0] * img.shape[1] * 0.5
        
        print("Gerando máscaras automáticas...")
        masks_data = self.auto_generator.generate(img)
        print(f"  {len(masks_data)} máscaras geradas")
        
        detections = []
        
        for mask_info in masks_data:
            mask = mask_info['segmentation']
            area = mask_info['area']
            bbox = mask_info['bbox']  # x, y, w, h formato COCO
            score = mask_info['predicted_iou']
            
            # Filtrar por área
            if area < min_area or area > max_area:
                continue
            
            # Filtrar por aspect ratio (fósseis são alongados)
            x, y, w, h = bbox
            aspect_ratio = max(w, h) / (min(w, h) + 1e-6)
            
            if aspect_ratio < aspect_ratio_range[0] or aspect_ratio > aspect_ratio_range[1]:
                continue
            
            # Converter bbox para x1,y1,x2,y2
            bbox_xyxy = (x, y, x + w, y + h)
            
            detections.append(Detection(
                bbox=bbox_xyxy,
                mask=mask,
                confidence=score
            ))
        
        # Ordenar por confiança
        detections.sort(key=lambda d: d.confidence, reverse=True)
        
        print(f"  {len(detections)} detecções após filtros")
        return detections

In [ ]:
# ============================================
# Exemplo SAM
# ============================================

# sam_detector = SAMDetector(
#     checkpoint_path="./models/sam_vit_b_01ec64.pth",
#     model_type="vit_b"
# )

# img = load_image("./sponge_images/GM1405_2.JPG")

# # Opção 1: Com ponto no centro do fóssil
# detections = sam_detector.segment_with_points(img, [(600, 400)], [1])

# # Opção 2: Com box aproximado
# detections = sam_detector.segment_with_box(img, (200, 300, 1000, 500))

# # Opção 3: Automático
# detections = sam_detector.segment_automatic(img)

# visualize_detections(img, detections, "SAM Detection")

---
## 3. Grounding DINO + SAM

Grounding DINO detecta objetos por descrição textual. Combinado com SAM, dá detecção + segmentação.

### Setup:
```bash
pip install groundingdino-py
# ou
git clone https://github.com/IDEA-Research/GroundingDINO.git
cd GroundingDINO && pip install -e .
```

In [ ]:
class GroundedSAMDetector:
    """
    Grounding DINO para detecção + SAM para segmentação
    """
    
    def __init__(self,
                 grounding_config: str = "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py",
                 grounding_checkpoint: str = "groundingdino_swint_ogc.pth",
                 sam_checkpoint: str = "sam_vit_b_01ec64.pth",
                 sam_type: str = "vit_b",
                 device: str = DEVICE):
        
        self.device = device
        
        # Carregar Grounding DINO
        try:
            from groundingdino.util.inference import load_model, predict
            from groundingdino.util import box_ops
            
            print("Carregando Grounding DINO...")
            self.grounding_model = load_model(grounding_config, grounding_checkpoint, device=device)
            self.grounding_predict = predict
            self.box_ops = box_ops
            self.grounding_available = True
            print("Grounding DINO carregado!")
        except Exception as e:
            print(f"Grounding DINO não disponível: {e}")
            self.grounding_available = False
        
        # Carregar SAM
        try:
            from segment_anything import sam_model_registry, SamPredictor
            
            print("Carregando SAM...")
            sam = sam_model_registry[sam_type](checkpoint=sam_checkpoint)
            sam.to(device)
            self.sam_predictor = SamPredictor(sam)
            self.sam_available = True
            print("SAM carregado!")
        except Exception as e:
            print(f"SAM não disponível: {e}")
            self.sam_available = False
    
    def detect(self,
               img: np.ndarray,
               text_prompt: str = "fossil . sponge . elongated organism . paleontological specimen",
               box_threshold: float = 0.25,
               text_threshold: float = 0.25) -> List[Detection]:
        """
        Detecta objetos baseado em descrição textual.
        
        Args:
            img: Imagem RGB
            text_prompt: Descrições separadas por " . "
            box_threshold: Threshold para detecção
            text_threshold: Threshold para matching de texto
        """
        if not self.grounding_available:
            print("Grounding DINO não disponível")
            return []
        
        from groundingdino.util.inference import load_image as gd_load_image
        import torchvision.transforms as T
        
        # Preparar imagem para Grounding DINO
        transform = T.Compose([
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        img_pil = Image.fromarray(img)
        img_transformed = transform(img_pil)
        
        # Detectar
        boxes, logits, phrases = self.grounding_predict(
            self.grounding_model,
            img_transformed,
            text_prompt,
            box_threshold,
            text_threshold,
            device=self.device
        )
        
        h, w = img.shape[:2]
        
        detections = []
        
        for box, score, phrase in zip(boxes, logits, phrases):
            # Converter de cxcywh normalizado para xyxy
            cx, cy, bw, bh = box.cpu().numpy()
            x1 = int((cx - bw/2) * w)
            y1 = int((cy - bh/2) * h)
            x2 = int((cx + bw/2) * w)
            y2 = int((cy + bh/2) * h)
            
            # Segmentar com SAM se disponível
            mask = None
            if self.sam_available:
                self.sam_predictor.set_image(img)
                masks, scores, _ = self.sam_predictor.predict(
                    box=np.array([x1, y1, x2, y2]),
                    multimask_output=False
                )
                mask = masks[0]
            
            detections.append(Detection(
                bbox=(x1, y1, x2, y2),
                mask=mask,
                confidence=float(score),
                label=phrase
            ))
        
        return detections

In [ ]:
# ============================================
# Exemplo Grounding DINO + SAM
# ============================================

# detector = GroundedSAMDetector(
#     grounding_checkpoint="./models/groundingdino_swint_ogc.pth",
#     sam_checkpoint="./models/sam_vit_b_01ec64.pth"
# )

# img = load_image("./sponge_images/GM1405_2.JPG")

# # Prompts variados para fósseis
# detections = detector.detect(
#     img,
#     text_prompt="fossil . orange elongated shape . preserved organism . sponge specimen",
#     box_threshold=0.2,
#     text_threshold=0.2
# )

# visualize_detections(img, detections, "Grounding DINO + SAM")

---
## 4. Claude Vision API

Usar Claude para detectar bounding boxes via prompt. Simples e não requer GPU local.

**Vantagens:**
- Não precisa de setup de modelos locais
- Entende contexto ("esponjas fósseis do Cambriano")
- Pode dar informações adicionais sobre o espécime

**Desvantagens:**
- Custo de API
- Latência
- Não gera máscaras de segmentação (apenas bboxes)

In [ ]:
import base64
import json
import re

class ClaudeVisionDetector:
    """
    Detector usando Claude Vision API
    """
    
    def __init__(self, api_key: str = None):
        try:
            import anthropic
            self.client = anthropic.Anthropic(api_key=api_key)
            self.available = True
        except Exception as e:
            print(f"Anthropic client não disponível: {e}")
            self.available = False
    
    def _image_to_base64(self, img: np.ndarray) -> str:
        """Converte imagem para base64"""
        # Redimensionar se muito grande (limite de 20MB)
        h, w = img.shape[:2]
        max_dim = 2000
        if max(h, w) > max_dim:
            scale = max_dim / max(h, w)
            img = cv2.resize(img, None, fx=scale, fy=scale)
        
        _, buffer = cv2.imencode('.jpg', cv2.cvtColor(img, cv2.COLOR_RGB2BGR), 
                                  [cv2.IMWRITE_JPEG_QUALITY, 85])
        return base64.b64encode(buffer).decode('utf-8')
    
    def detect(self, 
               img: np.ndarray,
               context: str = "Cambrian fossil sponges (Leptomitid)") -> List[Detection]:
        """
        Detecta fósseis usando Claude Vision.
        
        Args:
            img: Imagem RGB
            context: Contexto sobre o tipo de fóssil
        """
        if not self.available:
            print("Claude client não disponível")
            return []
        
        h, w = img.shape[:2]
        img_b64 = self._image_to_base64(img)
        
        prompt = f"""Analyze this paleontological photograph showing {context}.

Detect all fossil specimens in the image. For each fossil found, provide:
1. Bounding box coordinates as [x1, y1, x2, y2] in pixels
2. Confidence score (0-1)
3. Brief description

Image dimensions: {w} x {h} pixels

Respond ONLY with a JSON array in this exact format:
[
  {{
    "bbox": [x1, y1, x2, y2],
    "confidence": 0.95,
    "description": "elongated sponge with visible spicules"
  }}
]

If no fossils are found, return an empty array: []"""
        
        try:
            response = self.client.messages.create(
                model="claude-sonnet-4-20250514",
                max_tokens=1024,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "image",
                                "source": {
                                    "type": "base64",
                                    "media_type": "image/jpeg",
                                    "data": img_b64
                                }
                            },
                            {
                                "type": "text",
                                "text": prompt
                            }
                        ]
                    }
                ]
            )
            
            # Parsear resposta
            text = response.content[0].text
            
            # Extrair JSON
            json_match = re.search(r'\[.*\]', text, re.DOTALL)
            if json_match:
                results = json.loads(json_match.group())
            else:
                print(f"Não foi possível parsear resposta: {text}")
                return []
            
            detections = []
            for r in results:
                bbox = tuple(int(x) for x in r['bbox'])
                detections.append(Detection(
                    bbox=bbox,
                    mask=None,
                    confidence=r.get('confidence', 0.8),
                    label=r.get('description', 'fossil')
                ))
            
            return detections
            
        except Exception as e:
            print(f"Erro na API: {e}")
            return []
    
    def detect_and_segment(self,
                           img: np.ndarray,
                           sam_detector: 'SAMDetector',
                           context: str = "Cambrian fossil sponges") -> List[Detection]:
        """
        Claude para detecção + SAM para segmentação.
        Melhor dos dois mundos!
        """
        # Claude detecta bboxes
        detections = self.detect(img, context)
        
        if not detections:
            return []
        
        # SAM segmenta cada bbox
        refined_detections = []
        
        for det in detections:
            sam_results = sam_detector.segment_with_box(img, det.bbox)
            if sam_results:
                sam_det = sam_results[0]
                refined_detections.append(Detection(
                    bbox=sam_det.bbox,
                    mask=sam_det.mask,
                    confidence=det.confidence,
                    label=det.label
                ))
            else:
                refined_detections.append(det)
        
        return refined_detections

In [ ]:
# ============================================
# Exemplo Claude Vision
# ============================================

# import os
# os.environ["ANTHROPIC_API_KEY"] = "sua-api-key"

# claude_detector = ClaudeVisionDetector()
# img = load_image("./sponge_images/GM1405_2.JPG")

# # Apenas detecção
# detections = claude_detector.detect(img, context="Leptomitid sponges from Burgess Shale")

# # Ou detecção + segmentação com SAM
# # detections = claude_detector.detect_and_segment(img, sam_detector)

# visualize_detections(img, detections, "Claude Vision")

---
## 5. YOLOv8 - Fine-tuning com Poucas Amostras

Se você tiver algumas anotações (~20-50 imagens), pode treinar YOLO rapidamente.

### Workflow:
1. Anotar imagens com LabelImg ou Roboflow
2. Fine-tune YOLOv8 pré-treinado
3. Inferência

In [ ]:
class YOLODetector:
    """
    YOLOv8 para detecção de fósseis.
    Pode usar modelo pré-treinado ou fine-tuned.
    """
    
    def __init__(self, model_path: str = "yolov8n.pt"):
        try:
            from ultralytics import YOLO
            self.model = YOLO(model_path)
            self.available = True
            print(f"YOLOv8 carregado: {model_path}")
        except Exception as e:
            print(f"YOLOv8 não disponível: {e}")
            self.available = False
    
    def detect(self, 
               img: np.ndarray,
               conf_threshold: float = 0.25) -> List[Detection]:
        """
        Detecta objetos na imagem.
        
        Nota: Com modelo genérico, vai detectar classes do COCO.
        Para fósseis, precisa fine-tuning.
        """
        if not self.available:
            return []
        
        results = self.model(img, conf=conf_threshold, verbose=False)
        
        detections = []
        
        for r in results:
            boxes = r.boxes
            if boxes is None:
                continue
                
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                label = self.model.names[cls]
                
                # Se tiver máscara (YOLO-seg)
                mask = None
                if hasattr(r, 'masks') and r.masks is not None:
                    mask = r.masks.data[0].cpu().numpy()
                
                detections.append(Detection(
                    bbox=(x1, y1, x2, y2),
                    mask=mask,
                    confidence=conf,
                    label=label
                ))
        
        return detections
    
    @staticmethod
    def create_dataset_yaml(data_dir: str, 
                            classes: List[str] = ["fossil_sponge"],
                            output_path: str = "dataset.yaml"):
        """
        Cria arquivo YAML para treinar YOLO.
        
        Estrutura esperada:
        data_dir/
        ├── images/
        │   ├── train/
        │   └── val/
        └── labels/
            ├── train/
            └── val/
        """
        yaml_content = f"""
path: {data_dir}
train: images/train
val: images/val

names:
"""
        for i, cls in enumerate(classes):
            yaml_content += f"  {i}: {cls}\n"
        
        with open(output_path, 'w') as f:
            f.write(yaml_content)
        
        print(f"Dataset YAML criado: {output_path}")
        return output_path
    
    def train(self,
              data_yaml: str,
              epochs: int = 50,
              imgsz: int = 640,
              batch: int = 8,
              output_dir: str = "./yolo_training"):
        """
        Fine-tune YOLO no dataset de fósseis.
        """
        if not self.available:
            print("YOLO não disponível")
            return
        
        results = self.model.train(
            data=data_yaml,
            epochs=epochs,
            imgsz=imgsz,
            batch=batch,
            project=output_dir,
            name="fossil_detector",
            pretrained=True,
            patience=10,  # Early stopping
            device=DEVICE
        )
        
        print(f"Treinamento completo! Modelo em: {output_dir}/fossil_detector/weights/best.pt")
        return results

In [ ]:
# ============================================
# Exemplo YOLOv8 Fine-tuning
# ============================================

# 1. Criar estrutura de dados (precisa anotar manualmente primeiro)
# YOLODetector.create_dataset_yaml(
#     data_dir="./fossil_dataset",
#     classes=["fossil_sponge", "matrix"],
#     output_path="fossil_dataset.yaml"
# )

# 2. Treinar
# yolo = YOLODetector("yolov8n.pt")  # Modelo nano (rápido)
# yolo.train("fossil_dataset.yaml", epochs=50)

# 3. Usar modelo treinado
# yolo_trained = YOLODetector("./yolo_training/fossil_detector/weights/best.pt")
# detections = yolo_trained.detect(img)

---
## 6. Pipeline Unificado

In [ ]:
class FossilDetectionPipeline:
    """
    Pipeline unificado para detecção de fósseis.
    Escolhe automaticamente o melhor método disponível.
    """
    
    def __init__(self,
                 sam_checkpoint: str = None,
                 grounding_checkpoint: str = None,
                 yolo_checkpoint: str = None,
                 use_claude: bool = False,
                 claude_api_key: str = None):
        
        self.detectors = {}
        
        # SAM
        if sam_checkpoint and Path(sam_checkpoint).exists():
            try:
                self.detectors['sam'] = SAMDetector(sam_checkpoint)
            except Exception as e:
                print(f"Erro ao carregar SAM: {e}")
        
        # YOLO
        if yolo_checkpoint:
            try:
                self.detectors['yolo'] = YOLODetector(yolo_checkpoint)
            except Exception as e:
                print(f"Erro ao carregar YOLO: {e}")
        
        # Claude
        if use_claude:
            try:
                self.detectors['claude'] = ClaudeVisionDetector(claude_api_key)
            except Exception as e:
                print(f"Erro ao inicializar Claude: {e}")
        
        print(f"Detectores disponíveis: {list(self.detectors.keys())}")
    
    def detect(self,
               img: np.ndarray,
               method: str = 'auto',
               **kwargs) -> List[Detection]:
        """
        Detecta fósseis usando o método especificado.
        
        Args:
            img: Imagem RGB
            method: 'auto', 'sam', 'yolo', 'claude', 'claude+sam'
        """
        if method == 'auto':
            # Prioridade: YOLO treinado > Claude+SAM > SAM auto
            if 'yolo' in self.detectors:
                method = 'yolo'
            elif 'claude' in self.detectors and 'sam' in self.detectors:
                method = 'claude+sam'
            elif 'sam' in self.detectors:
                method = 'sam'
            elif 'claude' in self.detectors:
                method = 'claude'
            else:
                print("Nenhum detector disponível!")
                return []
        
        print(f"Usando método: {method}")
        
        if method == 'sam' and 'sam' in self.detectors:
            return self.detectors['sam'].segment_automatic(img, **kwargs)
        
        elif method == 'yolo' and 'yolo' in self.detectors:
            return self.detectors['yolo'].detect(img, **kwargs)
        
        elif method == 'claude' and 'claude' in self.detectors:
            return self.detectors['claude'].detect(img, **kwargs)
        
        elif method == 'claude+sam':
            if 'claude' in self.detectors and 'sam' in self.detectors:
                return self.detectors['claude'].detect_and_segment(
                    img, self.detectors['sam'], **kwargs
                )
        
        print(f"Método {method} não disponível")
        return []
    
    def process_and_extract_windows(self,
                                    img: np.ndarray,
                                    method: str = 'auto',
                                    window_sizes: List[int] = [64, 128, 256],
                                    stride: int = 32,
                                    min_content_ratio: float = 0.5) -> Dict:
        """
        Detecta fósseis e extrai janelas de convolução.
        """
        detections = self.detect(img, method=method)
        
        all_windows = []
        
        for i, det in enumerate(detections):
            if det.mask is None:
                continue
            
            # Extrair ROI
            x1, y1, x2, y2 = det.bbox
            roi_img = img[y1:y2, x1:x2]
            roi_mask = det.mask[y1:y2, x1:x2]
            
            h, w = roi_img.shape[:2]
            
            # Extrair janelas
            for window_size in window_sizes:
                if window_size > min(h, w):
                    continue
                
                for y in range(0, h - window_size + 1, stride):
                    for x in range(0, w - window_size + 1, stride):
                        window_img = roi_img[y:y+window_size, x:x+window_size]
                        window_mask = roi_mask[y:y+window_size, x:x+window_size]
                        
                        content_ratio = np.sum(window_mask > 0) / (window_size ** 2)
                        
                        if content_ratio >= min_content_ratio:
                            all_windows.append({
                                'image': window_img,
                                'position': (x + x1, y + y1),
                                'window_size': window_size,
                                'fossil_id': i,
                                'content_ratio': content_ratio
                            })
        
        return {
            'detections': detections,
            'windows': all_windows
        }

---
## 7. Helper: Anotação Semi-Automática

In [ ]:
def create_annotation_from_detection(img_path: str,
                                     detections: List[Detection],
                                     output_dir: str,
                                     format: str = 'yolo'):
    """
    Converte detecções para formato de anotação YOLO.
    Útil para criar dataset de treino a partir de detecções automáticas.
    
    Args:
        img_path: Caminho da imagem
        detections: Lista de detecções
        output_dir: Diretório de saída
        format: 'yolo' ou 'coco'
    """
    import shutil
    
    img = load_image(img_path)
    h, w = img.shape[:2]
    
    output_path = Path(output_dir)
    images_dir = output_path / 'images' / 'train'
    labels_dir = output_path / 'labels' / 'train'
    
    images_dir.mkdir(parents=True, exist_ok=True)
    labels_dir.mkdir(parents=True, exist_ok=True)
    
    # Copiar imagem
    img_name = Path(img_path).name
    shutil.copy(img_path, images_dir / img_name)
    
    # Criar arquivo de labels
    label_file = labels_dir / (Path(img_path).stem + '.txt')
    
    with open(label_file, 'w') as f:
        for det in detections:
            x1, y1, x2, y2 = det.bbox
            
            # Converter para formato YOLO (normalizado)
            cx = ((x1 + x2) / 2) / w
            cy = ((y1 + y2) / 2) / h
            bw = (x2 - x1) / w
            bh = (y2 - y1) / h
            
            # class_id x_center y_center width height
            f.write(f"0 {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")
    
    print(f"Anotação criada: {label_file}")


def batch_create_annotations(image_dir: str,
                             pipeline: FossilDetectionPipeline,
                             output_dir: str,
                             method: str = 'auto'):
    """
    Cria anotações para todas as imagens de um diretório.
    Depois você pode revisar manualmente e corrigir.
    """
    image_paths = list(Path(image_dir).glob('*.jpg')) + \
                  list(Path(image_dir).glob('*.JPG')) + \
                  list(Path(image_dir).glob('*.png'))
    
    print(f"Processando {len(image_paths)} imagens...")
    
    for img_path in image_paths:
        try:
            img = load_image(str(img_path))
            detections = pipeline.detect(img, method=method)
            
            if detections:
                create_annotation_from_detection(
                    str(img_path), detections, output_dir
                )
                print(f"  {img_path.name}: {len(detections)} detecções")
            else:
                print(f"  {img_path.name}: nenhuma detecção")
                
        except Exception as e:
            print(f"  Erro em {img_path.name}: {e}")
    
    print(f"\nAnotações salvas em {output_dir}")
    print("Revise manualmente antes de treinar!")

---
## 8. Exemplo Completo

In [ ]:
# ============================================
# WORKFLOW RECOMENDADO
# ============================================

# OPÇÃO A: Sem treinamento (mais simples)
# -----------------------------------------
# 1. Usar Claude para detectar bboxes
# 2. SAM para segmentar dentro dos bboxes

# pipeline = FossilDetectionPipeline(
#     sam_checkpoint="./models/sam_vit_b_01ec64.pth",
#     use_claude=True
# )

# img = load_image("./sponge_images/GM1405_2.JPG")
# result = pipeline.process_and_extract_windows(img, method='claude+sam')

# print(f"Detectados: {len(result['detections'])} fósseis")
# print(f"Extraídas: {len(result['windows'])} janelas")

# visualize_detections(img, result['detections'], "Pipeline Completo")

In [ ]:
# OPÇÃO B: Com fine-tuning YOLO (mais robusto)
# -----------------------------------------
# 1. Usar Claude+SAM para gerar anotações iniciais
# 2. Revisar manualmente
# 3. Treinar YOLO
# 4. Usar YOLO treinado

# # Passo 1: Gerar anotações
# pipeline = FossilDetectionPipeline(sam_checkpoint="./models/sam_vit_b_01ec64.pth", use_claude=True)
# batch_create_annotations("./raw_images", pipeline, "./fossil_dataset", method='claude+sam')

# # Passo 2: REVISAR MANUALMENTE AS ANOTAÇÕES!
# # Use LabelImg ou Roboflow

# # Passo 3: Treinar
# YOLODetector.create_dataset_yaml("./fossil_dataset", ["fossil_sponge"])
# yolo = YOLODetector("yolov8n.pt")
# yolo.train("dataset.yaml", epochs=50)

# # Passo 4: Usar
# trained_yolo = YOLODetector("./yolo_training/fossil_detector/weights/best.pt")
# detections = trained_yolo.detect(img)

---
## Comparação dos Métodos

| Método | Requer GPU? | Requer Treino? | Precisão | Setup |
|--------|-------------|----------------|----------|-------|
| SAM (auto) | Recomendado | Não | Média* | Médio |
| SAM (prompt) | Recomendado | Não | Alta | Fácil |
| Grounding DINO + SAM | Sim | Não | Alta | Complexo |
| Claude Vision | Não | Não | Boa | Muito fácil |
| Claude + SAM | Recomendado | Não | Alta | Fácil |
| YOLOv8 fine-tuned | Sim | Sim (~50 imgs) | Muito alta | Médio |

*SAM automático precisa de filtros para selecionar fósseis vs background

### Recomendação:

1. **Começar com**: Claude + SAM (sem setup, boa precisão)
2. **Para produção**: Treinar YOLOv8 com anotações geradas semi-automaticamente

---
## Downloads dos Modelos

```bash
# Criar diretório
mkdir -p models && cd models

# SAM ViT-B (375MB) - recomendado para começar
wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

# SAM ViT-H (2.4GB) - mais preciso
wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

# Grounding DINO (694MB)
wget https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
```